In [1]:
from datasets import concatenate_datasets, load_from_disk, DatasetDict

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileInfo size changed, may indicate binary incompatibility. Expected 64 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileSelector size changed, may indicate binary incompatibility. Expected 48 from C header, got 72 from PyObject


In [2]:
article_classification_task_dataset = load_from_disk('../datasets/tasks/article_classification_task_dataset/').shuffle(seed=42)
institution_classification_task_dataset = load_from_disk('../datasets/tasks/institution_classification_task_dataset/').shuffle(seed=42)
keyword_generation_task_dataset = load_from_disk('../datasets/tasks/keyword_generation_task_dataset/').shuffle(seed=42)
people_classification_task_dataset = load_from_disk('../datasets/tasks/people_classification_task_dataset/').shuffle(seed=42)

In [3]:
keyword_generation_task_dataset['train']['text'][0]

'[címkék generálása]\nSaját nevén publikálta a fiktív pályázat anyagát az egyetemi vezető\n\nA nemzetvédelmi egyetem kancellárja saját nevén egy folyóiratban még publikálta is annak a tanulmánynak az összefoglalóját, amelyet egy fiktív pályázat ala\n\nA nemzetvédelmi egyetem kancellárja saját nevén egy folyóiratban még publikálta is annak a tanulmánynak az összefoglalóját, amelyet egy fiktív pályázat alapján egy pesterzsébeti piacon grillcsirkét árusító vállalkozó nyert el, de valójában az egyetem munkatársa készített el – értesült hétfőn az MTI Hírcentrum.\nAz MR1-Kossuth Rádió Krónika című műsora vasárnap számolt be arról, hogy a Zrínyi Miklós Nemzetvédelmi Egyetem (ZMNE) gazdasági átvilágításakor kiderült: 2010 februárjában névlegesen a pesterzsébeti piacon grillcsirkét árusító vállalkozó végzett szociológiai kutatást a honvédségnél a ZMNE számára, amelyért nyolcmillió forintot fizettek ki.\nAz m1 hétfő esti Híradójában azt mondták: mostanra kiderült, hogy nemcsak a közbeszerzési fe

In [4]:
def get_pandas(dataset):
    return {split: dataset[split].to_pandas() for split in dataset.keys()}

In [5]:
pandas_article_classification_task_dataset = get_pandas(article_classification_task_dataset)
pandas_institution_classification_task_dataset = get_pandas(institution_classification_task_dataset)
pandas_keyword_generation_task_dataset = get_pandas(keyword_generation_task_dataset)
pandas_people_classification_task_dataset = get_pandas(people_classification_task_dataset)

In [6]:
for split in article_classification_task_dataset.keys():
    print(split, pandas_article_classification_task_dataset[split].groupby(['label']).size(), sep='\n', end='\n\n')

test
label
0    1898
1    1720
dtype: int64

train
label
0    16399
1    26661
dtype: int64

validation
label
0    3188
1    3410
dtype: int64



In [7]:
institution_classification_task_dataset

DatasetDict({
    test: Dataset({
        features: ['text', 'type'],
        num_rows: 494
    })
    train: Dataset({
        features: ['text', 'type'],
        num_rows: 7191
    })
    validation: Dataset({
        features: ['text', 'type'],
        num_rows: 919
    })
})

In [8]:
keyword_generation_task_dataset

DatasetDict({
    validation: Dataset({
        features: ['text', 'type'],
        num_rows: 3397
    })
    test: Dataset({
        features: ['text', 'type'],
        num_rows: 1710
    })
    train: Dataset({
        features: ['text', 'type'],
        num_rows: 26534
    })
})

In [9]:
people_classification_task_dataset

DatasetDict({
    test: Dataset({
        features: ['text', 'type'],
        num_rows: 750
    })
    train: Dataset({
        features: ['text', 'type'],
        num_rows: 11417
    })
    validation: Dataset({
        features: ['text', 'type'],
        num_rows: 1448
    })
})

In [10]:
combined_tasks = DatasetDict()

In [11]:
#combined_tasks = keyword_generation_task_dataset

In [12]:
#for split, size in zip(['train', 'validation', 'test'], [16000, 3000, 1500]):
#    combined_tasks[split] = article_classification_task_dataset[split].filter(lambda article: article['label'] == 1).select(range(size//10))
#    combined_tasks[split] = concatenate_datasets([combined_tasks[split], article_classification_task_dataset[split].filter(lambda article: article['label'] == 0).select(range(size))])
#combined_tasks = combined_tasks.remove_columns(['label', 'type'])
for split, size in zip(['train', 'validation', 'test'], [11000, 1400, 750]):
    combined_tasks[split] = people_classification_task_dataset[split].select(range(size)).remove_columns(['type'])
for split, size in zip(['train', 'validation', 'test'], [7100, 900, 490]):
    combined_tasks[split] = concatenate_datasets([combined_tasks[split], institution_classification_task_dataset[split].select(range(size)).remove_columns(['type'])])
for split, size in zip(['train', 'validation', 'test'], [10000, 3000, 1500]):
    combined_tasks[split] = concatenate_datasets([combined_tasks[split], keyword_generation_task_dataset[split].select(range(size)).remove_columns(['type'])])

In [13]:
combined_tasks.shuffle(seed=42).save_to_disk('../datasets/tasks/combined_task_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/28100 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5300 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2740 [00:00<?, ? examples/s]

In [14]:
n = 1

In [15]:
n += 1
print(combined_tasks.shuffle(seed=42)['train'][n]['text'])

[személy klasszifikáció]
Tavaly októberben írtuk, hogy ismét Török Márkot választotta a szegedi egyetem hallgatói önkormányzata (EHÖK) elnökké. Török olyan régóta tölti be ezt a posztot, hogy 2015-ben gyakorlatilag rá (is) szabtak egy korlátozó törvénymódosítást, ami legfeljebb négy évben szabja meg, hogy mennyit lehet valaki HÖK-elnök. Az egyetem meg is semmisítette a megválasztását, majd az egyetem 12 dékánja is felszólította őt távozásra. Török erre támadó válaszlevelet írt.
Ő egyébként úgy értelmezte a törvénymódosítást, hogy az nem visszaható, azaz csak a hatálybalépéstől kell számítani a négy évet. Emiatt bírósági ügy is lett az egészből. A helyzetről nyáron készítettünk interjút Törökkel, ez itt olvasható. Az Átlátszó most azt írja, a bíróság első fokon nem osztotta Török jogértelmezését, és úgy döntött, Török Márk nem lehet a Szegedi Tudományegyetem hallgatói önkormányzatának elnöke. A bíróság szerint a 2015-ben törvénybe iktatott négyéves felső korlát vonatkozik a tizenkét éve

In [16]:
#combined_tasks = load_from_disk('../datasets/tasks/combined_task_dataset/')

In [17]:
combined_tasks

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 28100
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 5300
    })
    test: Dataset({
        features: ['text'],
        num_rows: 2740
    })
})

In [18]:
combined_tasks_io = combined_tasks.map(lambda task: {
    'input': task['text'][:task['text'].rfind(':')+1],
    'output': task['text'][task['text'].rfind(':')+1:],
}, remove_columns='text')

Map:   0%|          | 0/28100 [00:00<?, ? examples/s]

Map:   0%|          | 0/5300 [00:00<?, ? examples/s]

Map:   0%|          | 0/2740 [00:00<?, ? examples/s]

In [19]:
print(combined_tasks_io['train'][1]['input'])

[személy klasszifikáció]
Talicskák, csontvázak, hűtlen kezelés, hivatali visszaélés, halállista, sorozatos szabálytalanságok, vádaskodások, hazudozások, perek – az esztergomi polgármesteri hivatalban és környékén az őszi választás óta "zajlik az élet", ám a helyzet feltehetőleg most tovább "fokozódik". Az Állami Számvevőszék a napokban nyilvánosságra hozta a korábbi városvezetés pénzügyi tevékenységét vizsgáló jelentését. Tétényi Éva polgármesterrel az ÁSZ-jelentés közzététele utáni új helyzetről beszélgettünk.
A 60 oldalas dokumentum szerint Meggyes Tamás, a város előző polgármestere, valamint Marosi György volt jegyző az elmúlt négy évben több alkalommal is törvényt sértett, ám ezek mindig következmények nélkül maradtak.
Tétényi Éva
A dokumentumból az is megtudható, hogy eközben a felelőtlen gazdálkodásnak köszönhetően elképesztő mértékűre nőtt az önkormányzat adósságterhe, s ennek "rendezése" alighanem komoly nehézségeket okoz majd nemcsak a jelenlegi vezetésnek, de a város lakosság

In [20]:
combined_tasks_io

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 28100
    })
    validation: Dataset({
        features: ['input', 'output'],
        num_rows: 5300
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 2740
    })
})

In [21]:
#combined_tasks_io.filter(lambda task: 'korrupciós címkék:' not in task['input']).save_to_disk('datasets/tasks/combined_task_dataset_io_notags')

In [22]:
combined_tasks_io.shuffle(seed=42).save_to_disk('../datasets/tasks/combined_people_institution')

Saving the dataset (0/1 shards):   0%|          | 0/28100 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5300 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2740 [00:00<?, ? examples/s]

In [23]:
combined_tasks_io.shuffle(seed=42).filter(lambda task: '[címkék generálása]' in task['input'])

Filter:   0%|          | 0/28100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5300 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2740 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['input', 'output'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 1500
    })
})

In [24]:
combined_tasks_io.shuffle(seed=42).filter(lambda task: '[címkék generálása]' in task['input']).save_to_disk('datasets/tasks/combined_task_dataset_io_tags')

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1500 [00:00<?, ? examples/s]

In [25]:
#combined_tasks_io.filter(lambda task: '[korrupció klasszifikáció]' in task['input']).save_to_disk('../datasets/tasks/combined_task_dataset_io_notags')

In [26]:
combined_tasks_io.filter(lambda task: '[címkék generálása]' in task['input'])

Filter:   0%|          | 0/28100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5300 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2740 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['input', 'output'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 1500
    })
})